# Data Cleansing and Validation

## Introduction

In this activity, I will use input validation and label encoding to prepare a dataset for analysis. These are fundamental techniques used in all types of data analysis, from simple linear regression to complex neural networks.

In this activity, I will play the role of a data professional working for an investment firm that is attempting to invest in private companies with a valuation of at least $1 billion. These are often known as "unicorns." My client wants to develop a better understanding of unicorns, with the hope they can be early investors in future highly successful companies. They are particularly interested in the investment strategies of the three top unicorn investors: Sequoia Capital, Tiger Global Management, and Accel.

## Step 1: Imports

Import relevant Python libraries and packages: `numpy`, `pandas`, `seaborn`, and `pyplot` from `matplotlib`.

In [64]:
# Import libraries and packages.

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Load the dataset

The data contains details about unicorn companies, such as when they were founded, when they achieved unicorn status, and their current valuation. The dataset `Modified_Unicorn_Companies.csv` is loaded as `companies`.

In [65]:
# Run this cell so pandas displays all columns
pd.set_option('display.max_columns', None)

In [66]:
# RUN THIS CELL TO IMPORT YOUR DATA.
companies = pd.read_csv('/Users/gzuz/Repositories/GitHub/Data-Science-and-Analytics-Repository/Data Cleaning/Data/Modified_Unicorn_Companies.csv')

# Display the first five rows.
companies.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S..."
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China..."
3,Stripe,95,2014-01-23,FinTech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG"
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita..."


## Step 2: Data cleaning


Begin by displaying the data types of the columns in `companies`.

In [67]:
# Display the data types of the columns.

companies.dtypes

Company             object
Valuation            int64
Date Joined         object
Industry            object
City                object
Country/Region      object
Continent           object
Year Founded         int64
Funding             object
Select Investors    object
dtype: object

### Modify the data types

Notice that the data type of the `Date Joined` column is an `object`&mdash;in this case, a string. Convert this column to `datetime` to make it more usable.

In [68]:
# Apply necessary datatype conversions.

companies['Date Joined'] = pd.to_datetime(companies['Date Joined'])

### Create a new column

Add a column called `Years To Unicorn`, which is the number of years between when the company was founded and when it became a unicorn.

In [69]:
# Create the column Years To Unicorn.

companies['Years to unicorn'] = companies['Date Joined'].dt.year - companies['Year Founded']

**QUESTION: Why might a client be interested in how quickly a company achieved unicorn status?**

Knowing how quickly a company achieves unicorn status reveals alot about certain trends and commonalities present in the data. It can also give a lot of insight into different industries and how quickly they can grow.

### Input validation

Currently, the dataset contains some bad data, duplicate rows, and inconsistent `Industry` labels.

Identify and correct each of these issues.

#### Correcting bad data

Get descriptive statistics for the `Years To Unicorn` column.

In [70]:
# Identify and correct the issue with Years To Unicorn.

companies[['Years to unicorn']].describe()

,Years to unicorn
count,1074.000000
mean,7.013035
std,5.331842
min,-3.000000
25%,4.000000
50%,6.000000
75%,9.000000
max,98.000000


Isolate all rows where the `Years To Unicorn` column contains a negative value.

In [71]:
# Isolate any rows where `Years To Unicorn` is negative

mask = companies['Years to unicorn'] < 0

companies[mask]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years to unicorn
527,InVision,2,2017-11-01,Internet software & services,New York,United States,North America,2020,$349M,"FirstMark Capital, Tiger Global Management, IC...",-3


In [72]:
companies[companies['Years to unicorn'] < 0]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years to unicorn
527,InVision,2,2017-11-01,Internet software & services,New York,United States,North America,2020,$349M,"FirstMark Capital, Tiger Global Management, IC...",-3


**Question: How many rows have negative values in the `Years To Unicorn` column, and what companies are they for?**


Just one row and the company is called InVision

An internet search reveals that InVision was founded in 2011. Replace the value at `Year Founded` with `2011` for InVision's row.

In [73]:
# Replace InVision's `Year Founded` value with 2011
companies.loc[companies['Company'] == 'InVision', 'Year Founded'] = 2011

# Verify the change was made properly

companies[companies['Company'] == 'InVision']

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years to unicorn
527,InVision,2,2017-11-01,Internet software & services,New York,United States,North America,2011,$349M,"FirstMark Capital, Tiger Global Management, IC...",-3


Now, recalculate all the values in the `Years To Unicorn` column to remove the negative value for InVision. Verify that there are no more negative values afterwards.

In [74]:
# Recalculate all values in the `Years To Unicorn` column

companies['Years to unicorn'] = companies['Date Joined'].dt.year - companies['Year Founded']
# Verify that there are no more negative values in the column

companies['Years to unicorn'].describe()

count    1074.000000
mean        7.021415
std         5.323155
min         0.000000
25%         4.000000
50%         6.000000
75%         9.000000
max        98.000000
Name: Years to unicorn, dtype: float64

#### Issues with `Industry` labels

The company provided you with the following list of industry labels to identify in the data for `Industry`.

**Note:** Any labels in the `Industry` column that are not in `industry_list` are misspellings.

In [75]:
# List provided by the company of the expected industry labels in the data
industry_list = ['Artificial intelligence', 'Other','E-commerce & direct-to-consumer', 'Fintech',\
       'Internet software & services','Supply chain, logistics, & delivery', 'Consumer & retail',\
       'Data management & analytics', 'Edtech', 'Health', 'Hardware','Auto & transportation', \
        'Travel', 'Cybersecurity','Mobile & telecommunications']

First, check if there are values in the `Industry` column that are not in `industry_list`. If so, what are they?

In [76]:
# Check which values are in `Industry` but not in `industry_list`

set(companies['Industry']) - set(industry_list)

{'Artificial Intelligence', 'Data management and analytics', 'FinTech'}

**Question: Which values currently exist in the `Industry` column that are not in `industry_list`?**


All three values do not exist in the Indsutry dataframe due to mispellings

Now, correct the bad entries in the `Industry` column by replacing them with an approved string from `industry_list`. To do this, use the [`replace()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.replace.html) `Series` method on the `Industry` series. When you pass a dictionary to the method, it will replace the data in the series where that data matches the dictionary's keys. The values that get imputed are the values of the dictionary. If a value is not specified in the dictionary, the series' original value is retained.
</br></br>


1. Create a dictionary called `replacement_dict` whose keys are the incorrect spellings in the `Industry` series and whose values are the correct spelling, as indicated in `industry_list`.

2. Call the `replace()` method on the `Industry` series and pass to it `replacement_dict` as its argument. Reassign the result back to the `Industry` column.

3. Verify that there are no longer any elements in `Industry` that are not in `industry_list`.

In [77]:
# 1. Create `replacement_dict`

replacement_dict = {'Artificial Intelligence':'Artificial intelligence' , 'Data management and analytics':'Data management & analytics' , 'FinTech':'Fintech' }
replacement_dict
# 2. Replace the incorrect values in the `Industry` column

companies['Industry'] = companies['Industry'].replace(replacement_dict)
# 3. Verify that there are no longer any elements in `Industry` that are not in `industry_list`

set(companies['Industry']) - set(industry_list)

set()

#### Handling duplicate rows

The business mentioned that no company should appear in the data more than once.

Verify that this is indeed the case, and if not, clean the data so each company appears only once.

Begin by checking which, if any, companies are duplicated. Filter the data to return all occurrences of those duplicated companies.

In [78]:
# Isolate rows of all companies that have duplicates

companies[companies.duplicated(subset='Company', keep=False)]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years to unicorn
385,BrewDog,2,2017-04-10,Consumer & retail,Aberdeen,United Kingdom,Europe,2007,$233M,"TSG Consumer Partners, Crowdcube",10
386,BrewDog,2,2017-04-10,Consumer & retail,Aberdeen,UnitedKingdom,Europe,2007,$233M,TSG Consumer Partners,10
510,ZocDoc,2,2015-08-20,Health,New York,United States,North America,2007,$374M,"Founders Fund, Khosla Ventures, Goldman Sachs",8
511,ZocDoc,2,2015-08-20,Health,NaN,United States,North America,2007,$374M,Founders Fund,8
1031,SoundHound,1,2018-05-03,Artificial intelligence,Santa Clara,United States,North America,2005,$215M,"Tencent Holdings, Walden Venture Capital, Glob...",13
1032,SoundHound,1,2018-05-03,Other,Santa Clara,United States,North America,2005,$215M,Tencent Holdings,13


**Question: Do these duplicated companies seem like legitimate data points, or are they problematic data?**


These duplicates do seem like problematic data because the company names are completely the same with minor variations in other columns

Keep the first occurrence of each duplicate company and drop the subsequent rows that are copies.

In [79]:
# Drop rows of duplicate companies after their first occurrence

companies = companies.drop_duplicates(subset = 'Company', keep = 'first')

**Question: Why is it important to perform input validation?**


Input validation is important because it helps in ensuring the data is complete, and free of errors.

**Question: What steps did you take to perform input validation for this dataset?**


* Fixing incorrect values
* Correcting inconsistencies in the data
* Removing duplicate data

### Convert numerical data to categorical data

Sometimes, you'll want to simplify a numeric column by converting it to a categorical column. To do this, one common approach is to break the range of possible values into a defined number of equally sized bins and assign each bin a name. In the next step, you'll practice this process.

#### Create a `High Valuation` column

The data in the `Valuation` column represents how much money (in billions, USD) each company is valued at. Use the `Valuation` column to create a new column called `High Valuation`. For each company, the value in this column should be `low` if the company is in the bottom 50% of company valuations and `high` if the company is in the top 50%.

In [80]:
# Create new `High Valuation` column
# Use qcut to divide Valuation into 'high' and 'low' Valuation groups

companies['High Valuation'] = pd.qcut(companies['Valuation'], 2, labels = ['low', 'high'])

### Convert categorical data to numerical data

Three common methods for changing categorical data to numerical are:

1. Label encoding: order matters (ordinal numeric labels)
2. Label encoding: order doesn't matter (nominal numeric labels)
3. Dummy encoding: order doesn't matter (creation of binary columns for each possible category contained in the variable)

The decision on which method to use depends on the context and must be made on a case-to-case basis. However, a distinction is typically made between categorical variables with equal weight given to all possible categories vs. variables with a hierarchical structure of importance to their possible categories.  

For example, a variable called `subject` might have possible values of `history`, `mathematics`, `literature`. In this case, each subject might be **nominal**&mdash;given the same level of importance. However, you might have another variable called `class`, whose possible values are `freshman`, `sophomore`, `junior`, `senior`. In this case, the class variable is **ordinal**&mdash;its values have an ordered, hierarchical structure of importance.

Machine learning models typically need all data to be numeric, and they generally use ordinal label encoding (method 1) and dummy encoding (method 3).

In the next steps, I'll convert the following variables: `Continent`, `Country/Region`, and `Industry`, each using a different approach.

### Convert `Continent` to numeric

For the purposes of this exercise, suppose that the investment group has specified that they want to give more weight to continents with fewer unicorn companies because they believe this could indicate unrealized market potential.

**Question: Which type of variable would this make the `Continent` variable in terms of how it would be converted to a numeric data type?**


It would be an ordinal variable since there is greater importance being placed on continents with less unicorn companies.

Rank the continents in descending order from the greatest number of unicorn companies to the least.

In [81]:
# Rank the continents by number of unicorn companies

companies['Continent'].value_counts(ascending = False)

North America    586
Asia             310
Europe           143
South America     21
Oceania            8
Africa             3
Name: Continent, dtype: int64

Now, create a new column called `Continent Number` that represents the `Continent` column converted to numeric in the order of their number of unicorn companies, where North America is encoded as `1`, through Africa, encoded as `6`.

In [82]:
# Create numeric `Continent Number` column

cont_dict = {'North America': 1,
             'Asia': 2,
             'Europe': 3,
             'South America': 4,
             'Oceania': 5,
             'Africa': 6
            }

companies['Continent Number'] = companies['Continent'].replace(cont_dict)
companies.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years to unicorn,High Valuation,Continent Number
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S...",5,high,2
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10,high,1
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China...",10,high,2
3,Stripe,95,2014-01-23,Fintech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG",4,high,1
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita...",6,high,3


### Convert `Country/Region` to numeric

Now, suppose that within a given continent, each company's `Country/Region` is given equal importance. For analytical purposes, you want to convert the values in this column to numeric without creating a large number of dummy columns. Use label encoding of this nominal categorical variable to create a new column called `Country/Region Numeric`, wherein each unique `Country/Region` is assigned its own number.

In [83]:
# Create `Country/Region Numeric` column
# Create numeric categories for Country/Region

companies['Country/Region Numeric'] = companies['Country/Region'].astype('category').cat.codes
companies.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years to unicorn,High Valuation,Continent Number,Country/Region Numeric
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S...",5,high,2,9
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10,high,1,44
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China...",10,high,2,9
3,Stripe,95,2014-01-23,Fintech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG",4,high,1,44
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita...",6,high,3,38


### Convert `Industry` to numeric

Finally, create dummy variables for the values in the `Industry` column.

In [84]:
# Convert `Industry` to numeric data
# Create dummy variables with Industry values
encoded_industry = pd.get_dummies(companies['Industry'])

# Combine `companies` DataFrame with new dummy Industry columns

companies_encoded = pd.concat([companies, encoded_industry], axis=1)

Display the first few rows of `companies`

In [85]:
companies_encoded.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years to unicorn,High Valuation,Continent Number,Country/Region Numeric,Artificial intelligence,Auto & transportation,Consumer & retail,Cybersecurity,Data management & analytics,E-commerce & direct-to-consumer,Edtech,Fintech,Hardware,Health,Internet software & services,Mobile & telecommunications,Other,"Supply chain, logistics, & delivery",Travel
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S...",5,high,2,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10,high,1,44,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China...",10,high,2,9,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Stripe,95,2014-01-23,Fintech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG",4,high,1,44,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita...",6,high,3,38,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


**Question: Which categorical encoding approach did you use for each variable? Why?**

* For `Continent`, I used Ordinal label encoding because there was a hierarchical order to the categories.
* For `Country/Region`, I used Nominal label encoding because there wasn't a hierarchical order the categories.
* For `Industry`, I used Dummy encoding because there weren't many different categories and they were all equally important.

**Question: How does label encoding change the data?**


>Label encoding changes the data because it assigns unique numbers to categorical data which is easier for processing.

**Question: What are the benefits of label encoding?**


>Label encoding is used for machine learning models which require numerical input.

**Question: What are the disadvantages of label encoding?**


>It may make it difficult to interpret what a column value represents. Additionally, it may create unwanted realtionships between categorical data due to the numerical values, which can be misunderstood as hierachical rather than aliases.

## Conclusion

**What are some key takeaways that you learned during this lab?**

- Validating input is crucial for maintaining the integrity and accuracy of data.
- Input validation involves a process of experimentation to pinpoint problems and figure out optimal solutions.
- Both label encoding and dummy/one-hot encoding come with their own set of advantages and drawbacks.
- Choosing between label encoding and dummy/one-hot encoding should be done considering the specifics of each case.

**Reference**

[Bhat, M.A. *Unicorn Companies*](https://www.kaggle.com/datasets/mysarahmadbhat/unicorn-companies)

